In [65]:
from src.data import data, analysis


DATA_DIR = "data"
df = data.load_data(DATA_DIR)

In [52]:
# Take only a few liens for the development
df = df.iloc[:1]

In [66]:
from src.data.analysis import get_img_name

def get_imgs_path(productids: list[int], imageids: list[int]):
    if(len(productids) != len(imageids)):
        raise ValueError("productids and imageids should be the same size") 
    
    return [get_img_name(productid, imageid) for productid, imageid in zip(productids, imageids)]

filenames = get_imgs_path(df.loc[:, "productid"], df.loc[:, "imageid"])

In [68]:
import os
import time
import datetime
from PIL import Image, ImageOps
import numpy as np
from IPython.display import clear_output


class Progression():
    def __init__(self, total_rows:int, detection:int = 1000):
        self.start_time = time.perf_counter()
        self.total_rows = total_rows
        self.detection = detection

    def display(self, current_row_number: int):
        if current_row_number == 0:
            return
        
        if current_row_number % self.detection == 0:
            time_diff = time.perf_counter() - self.start_time
            time_per_row = time_diff / current_row_number
            remaining_time = (self.total_rows - current_row_number) * time_per_row

            clear_output(wait=True)
            print("Avancement : ", np.round(current_row_number/self.total_rows*100, 2), "%")
            print("Temps restant :", datetime.timedelta(seconds=int(remaining_time)))


imput_img_dir = "data/images/image_train/"
output_img_dir = "data/images/cropped_50/"
image_size = (50, 50)
keep_ratio = True

if not os.path.exists(output_img_dir):
    os.makedirs(output_img_dir)

nb_total_imgs = len(filenames)
progress = Progression(nb_total_imgs, detection=100)
for i, filename in enumerate(filenames):

    progress.display(i)

    img_array = np.asarray(Image.open(imput_img_dir + filename))
    top_line = -1
    right_line = -1
    bottom_line = -1
    left_line = -1

    i = 1
    can_continue = True
    while can_continue:
        if top_line == -1 and img_array[:i].mean() != 255:
            top_line = i
        if bottom_line == -1 and img_array[-i:].mean() != 255:
            bottom_line = i
        if left_line == -1 and img_array[:, :i].mean() != 255:
            left_line = i
        if right_line == -1 and img_array[:, -i:].mean() != 255:
            right_line = i

        if (top_line != -1 and bottom_line != -1
            and left_line != -1 and right_line != -1):
            can_continue = False
        
        i += 1

    new_img_array = img_array[top_line:-bottom_line, 
                              left_line:-right_line,
                              :]
    new_img = Image.fromarray(new_img_array)

    if keep_ratio:
        new_width = new_img.width
        new_height = new_img.height

        ratio = new_width - new_height
        padding_value = np.abs(ratio) // 2
        padding = ()
        if ratio > 0:
            padding = (0, padding_value, 0, padding_value)
        else:
            padding = (padding_value, 0, padding_value, 0)
            
        new_img = ImageOps.expand(new_img, padding, (255, 255, 255))

    new_img = new_img.resize(image_size)
    new_img.save(output_img_dir + filename)


Avancement :  1.18 %
Temps restant : 0:27:18


KeyboardInterrupt: 